<a href="https://colab.research.google.com/github/gaurav-awasthi/SoC2021/blob/main/Age_and_Gender_from_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# To install libraries required for loading youtube videos
!pip install pafy
!pip install youtube-dl

In [32]:
import os
# folder where pretrained data is present
path = '/content/drive/MyDrive/Colab Notebooks/My_Pretrained'

# location of output video
output_dir = '/content/drive/MyDrive/Colab Notebooks'

# pafy is for youtube video processing and time library
import cv2
import numpy as np
import pafy
import time

# in colab, use cv2_imshow instead of cv2.imshow
from google.colab.patches import cv2_imshow

In [34]:
# Number of seconds the video is run for
n = 5

# Setting up output video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
w, h = 1280,720
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
output_fps = 40
output_movie = cv2.VideoWriter(os.path.join(output_dir,'output_video.avi'), fourcc, output_fps, (w, h))

# URL of the video
url = 'https://youtu.be/YCoQwZ9BQ9Q'
vPafy = pafy.new(url)
play = vPafy.getbest(preftype="mp4")

# Capturing youtube video
cap = cv2.VideoCapture(play.url)

cap.set(3, 480) # width of the frame
cap.set(4, 640) # height of the frame

# Pretrained model mean values
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# age range has to be same as that of pre-trained model
age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']

gender_list = ['Male', 'Female']

# load models
def load_caffe_models():
 
  age_net = cv2.dnn.readNetFromCaffe(os.path.join(path,'deploy_age.prototxt'), os.path.join(path,'age_net.caffemodel'))

  gender_net = cv2.dnn.readNetFromCaffe(os.path.join(path,'deploy_gender.prototxt'), os.path.join(path,'gender_net.caffemodel'))

  return (age_net, gender_net)

# detecting age and gender
def video_detector(age_net, gender_net):
  # font for text in processed video
  font = cv2.FONT_HERSHEY_PLAIN
  framecount = 0

  # loop should end when FPS*n frames have been run
  while framecount < n*output_fps:

      ret, image = cap.read()
      framecount = framecount+1

      face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
      eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

      faces = face_cascade.detectMultiScale(gray, 1.1, 5)

      for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 255, 0), 2)

      #face in square 
        face_img = image[y:y+h, h:h+w].copy()
        blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

      # predicting gender
        gender_net.setInput(blob)
        gender_preds = gender_net.forward()
        gender = gender_list[gender_preds[0].argmax()]
        #print("Gender : " + gender)

      # perdicting age
        age_net.setInput(blob)
        age_preds = age_net.forward()
        age = age_list[age_preds[0].argmax()]
        #print("Age Range: " + age)

        overlay_text = "%s %s" % (gender, age)
        cv2.putText(image, overlay_text, (x, y), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
      output_movie.write(image)
  output_movie.release()
  print(framecount)


age_net, gender_net = load_caffe_models()

video_detector(age_net, gender_net)

200
